## Part 1: Scrap clean the data from the source and combine into one table.

In [1]:
import urllib.request
url_neigh='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url_school='https://en.wikipedia.org/wiki/List_of_secondary_schools_in_the_Toronto_District_School_Board'
url_demo='https://en.wikipedia.org/wiki/Demographics_of_Toronto'

page_neigh=urllib.request.urlopen(url_neigh)
page_school=urllib.request.urlopen(url_school)
page_demo=urllib.request.urlopen(url_demo)

In [2]:
page_neigh

In [3]:
page_school

In [4]:
page_demo

In [5]:
from bs4 import BeautifulSoup
soup_neigh=BeautifulSoup(page_neigh,"lxml")
print(soup_neigh.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"b6928b0a-c33c-4c16-9dca-a680f600278b","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":979555370,"wgRevisionId":979555370,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communicati

In [6]:
table_neigh=soup_neigh.find('table',class_='wikitable sortable')
table_neigh

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

In [7]:
A=[]
B=[]
C=[]

for row in table_neigh.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [8]:
import pandas as pd
import numpy as np

df_neigh = pd.DataFrame(A, columns = ['Postal Code'])
df_neigh['Borough'] = B
df_neigh['Neighbourhood'] = C
df_neigh

,Postal Code,Borough,Neighbourhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
...,...,...,...
175,M5Z\n,Not assigned\n,Not assigned\n
176,M6Z\n,Not assigned\n,Not assigned\n
177,M7Z\n,Not assigned\n,Not assigned\n
178,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."


In [9]:
df_neigh1 = df_neigh.replace('\n','', regex = True)
df_neigh1['Borough'] = df_neigh1['Borough'].replace('Not assigned', np.nan)
df_neigh2 = df_neigh1.dropna()
df_neigh2.reset_index(drop = True)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [10]:
df_neigh3 = df_neigh2.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(list)
df_neigh3 = df_neigh3.sample(frac = 1).reset_index()
df_neigh3['Neighbourhood'] = df_neigh3['Neighbourhood'].str.join(',')
df_neigh3

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1H,Scarborough,Cedarbrae
2,M5H,Downtown Toronto,"Richmond, Adelaide, King"
3,M6B,North York,Glencairn
4,M9L,North York,Humber Summit
...,...,...,...
98,M6R,West Toronto,"Parkdale, Roncesvalles"
99,M1S,Scarborough,Agincourt
100,M4W,Downtown Toronto,Rosedale
101,M2P,North York,York Mills West


In [11]:
df_neigh3['Neighbourhood'] = df_neigh3['Neighbourhood'].replace('Not assigned', df_neigh['Borough'])
df_neigh3

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1H,Scarborough,Cedarbrae
2,M5H,Downtown Toronto,"Richmond, Adelaide, King"
3,M6B,North York,Glencairn
4,M9L,North York,Humber Summit
...,...,...,...
98,M6R,West Toronto,"Parkdale, Roncesvalles"
99,M1S,Scarborough,Agincourt
100,M4W,Downtown Toronto,Rosedale
101,M2P,North York,York Mills West


In [12]:
coordinate_info = pd.read_csv('http://cocl.us/Geospatial_data')
coordinate_info = coordinate_info.set_index('Postal Code')
coordinate_info

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
...,...,...
M9N,43.706876,-79.518188
M9P,43.696319,-79.532242
M9R,43.688905,-79.554724


In [13]:
df_neigh4 = df_neigh3.set_index(['Postal Code'])
df_neigh4

,Borough,Neighbourhood
Postal Code,,
M1B,Scarborough,"Malvern, Rouge"
M1H,Scarborough,Cedarbrae
M5H,Downtown Toronto,"Richmond, Adelaide, King"
M6B,North York,Glencairn
M9L,North York,Humber Summit
...,...,...
M6R,West Toronto,"Parkdale, Roncesvalles"
M1S,Scarborough,Agincourt
M4W,Downtown Toronto,Rosedale


In [14]:
df_neigh5 = df_neigh4.merge(coordinate_info, how = 'left', left_index = True, right_index = True)
df_neigh5

,Borough,Neighbourhood,Latitude,Longitude
Postal Code,,,,
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
M6B,North York,Glencairn,43.709577,-79.445073
M9L,North York,Humber Summit,43.756303,-79.565963
...,...,...,...,...
M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
M1S,Scarborough,Agincourt,43.794200,-79.262029
M4W,Downtown Toronto,Rosedale,43.679563,-79.377529


In [15]:
df_neigh6=df_neigh5.set_index('Borough')
df_neigh6

,Neighbourhood,Latitude,Longitude
Borough,,,
Scarborough,"Malvern, Rouge",43.806686,-79.194353
Scarborough,Cedarbrae,43.773136,-79.239476
Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
North York,Glencairn,43.709577,-79.445073
North York,Humber Summit,43.756303,-79.565963
...,...,...,...
West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
Scarborough,Agincourt,43.794200,-79.262029
Downtown Toronto,Rosedale,43.679563,-79.377529


### 1.1 Repeat for the list of schools

In [16]:
soup_school=BeautifulSoup(page_school,"lxml")
print(soup_school.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of secondary schools in the Toronto District School Board - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"65749397-81cc-4ff5-890a-b440ab452ffb","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_secondary_schools_in_the_Toronto_District_School_Board","wgTitle":"List of secondary schools in the Toronto District School Board","wgCurRevisionId":959296707,"wgRevisionId":959296707,"wgArticleId":39477220,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1

In [17]:
table_school=soup_school.find('table',class_='wikitable sortable')
table_school

<table class="wikitable sortable">
<tbody><tr>
<th scope="col">Name<sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[5]</a></sup>
</th>
<th scope="col">Location
</th>
<th scope="col">Population (Dec. 2017)
</th>
<th scope="col">EQAO Academic Math score (2016-2018)
</th>
<th scope="col">ESL population
</th>
<th scope="col">OSSLT pass (2016-2018)
</th>
<th scope="col">Special programs<sup class="reference" id="cite_ref-6"><a href="#cite_note-6">[6]</a></sup><sup class="reference" id="cite_ref-7"><a href="#cite_note-7">[7]</a></sup>
</th>
<th>Picture
</th></tr>
<tr>
<td><a class="mw-redirect" href="/wiki/A._Y._Jackson_Secondary_School_(Toronto,_Ontario)" title="A. Y. Jackson Secondary School (Toronto, Ontario)">A. Y. Jackson Secondary School</a>
</td>
<td>North York
</td>
<td><span data-sort-value="7003105500000000000♠">1,055</span>
</td>
<td>93% <img alt="Increase" data-file-height="300" data-file-width="300" decoding="async" height="11" src="//upload.wikimedia.org/wikipedia

In [18]:
A = []
B = []
#C = []

for row in table_school.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) == 8:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
#        C.append(cells[6].find(text=True))

In [19]:
df_school = pd.DataFrame(A, columns = ['Name'])
df_school['Location'] = B
#df_school['Special Programs'] = C
df_school

,Name,Location
0,A. Y. Jackson Secondary School,North York\n
1,Agincourt Collegiate Institute,Scarborough\n
2,Albert Campbell Collegiate Institute,Scarborough\n
3,Birchmount Park Collegiate Institute,Scarborough\n
4,Bloor Collegiate Institute,Toronto\n
...,...,...
64,William Lyon Mackenzie Collegiate Institute,North York\n
65,Winston Churchill Collegiate Institute,Scarborough\n
66,Woburn Collegiate Institute,Scarborough\n
67,York Memorial Collegiate Institute,York\n


In [20]:
df_school1 = df_school.replace('\n','', regex = True)
df_school1

,Name,Location
0,A. Y. Jackson Secondary School,North York
1,Agincourt Collegiate Institute,Scarborough
2,Albert Campbell Collegiate Institute,Scarborough
3,Birchmount Park Collegiate Institute,Scarborough
4,Bloor Collegiate Institute,Toronto
...,...,...
64,William Lyon Mackenzie Collegiate Institute,North York
65,Winston Churchill Collegiate Institute,Scarborough
66,Woburn Collegiate Institute,Scarborough
67,York Memorial Collegiate Institute,York


In [21]:
# Combine schools with the neighbourhoods
df_school1['Name']=df_school1.groupby(['Location'])['Name'].transform(lambda x:','.join(x))
df_school1

,Name,Location
0,"A. Y. Jackson Secondary School,C. W. Jefferys ...",North York
1,"Agincourt Collegiate Institute,Albert Campbell...",Scarborough
2,"Agincourt Collegiate Institute,Albert Campbell...",Scarborough
3,"Agincourt Collegiate Institute,Albert Campbell...",Scarborough
4,"Bloor Collegiate Institute,Central Technical S...",Toronto
...,...,...
64,"A. Y. Jackson Secondary School,C. W. Jefferys ...",North York
65,"Agincourt Collegiate Institute,Albert Campbell...",Scarborough
66,"Agincourt Collegiate Institute,Albert Campbell...",Scarborough
67,"George Harvey Collegiate Institute,Runnymede C...",York


In [22]:
df_school2=df_school1.drop_duplicates()
df_school2

,Name,Location
0,"A. Y. Jackson Secondary School,C. W. Jefferys ...",North York
1,"Agincourt Collegiate Institute,Albert Campbell...",Scarborough
4,"Bloor Collegiate Institute,Central Technical S...",Toronto
15,"East York Collegiate Institute,Leaside High Sc...",East York
17,"Etobicoke Collegiate Institute,Kipling Collegi...",Etobicoke
20,"George Harvey Collegiate Institute,Runnymede C...",York


In [23]:
df_school3=df_school2.set_index('Location')
df_school3

,Name
Location,
North York,"A. Y. Jackson Secondary School,C. W. Jefferys ..."
Scarborough,"Agincourt Collegiate Institute,Albert Campbell..."
Toronto,"Bloor Collegiate Institute,Central Technical S..."
East York,"East York Collegiate Institute,Leaside High Sc..."
Etobicoke,"Etobicoke Collegiate Institute,Kipling Collegi..."
York,"George Harvey Collegiate Institute,Runnymede C..."


In [24]:
df=df_school3.merge(df_neigh6,how='left',left_index=True,right_index=True)
df1=df.reset_index()
df2=df1.rename(columns={'index':'Borough'})
df3=df2.dropna()
df3

,Borough,Name,Neighbourhood,Latitude,Longitude
0,East York,"East York Collegiate Institute,Leaside High Sc...","Parkview Hill, Woodbine Gardens",43.706397,-79.309937
1,East York,"East York Collegiate Institute,Leaside High Sc...",Woodbine Heights,43.695344,-79.318389
2,East York,"East York Collegiate Institute,Leaside High Sc...",Thorncliffe Park,43.705369,-79.349372
3,East York,"East York Collegiate Institute,Leaside High Sc...","East Toronto, Broadview North (Old East York)",43.685347,-79.338106
4,East York,"East York Collegiate Institute,Leaside High Sc...",Leaside,43.709060,-79.363452
...,...,...,...,...,...
59,York,"George Harvey Collegiate Institute,Runnymede C...","Runnymede, The Junction North",43.673185,-79.487262
60,York,"George Harvey Collegiate Institute,Runnymede C...",Weston,43.706876,-79.518188
61,York,"George Harvey Collegiate Institute,Runnymede C...",Humewood-Cedarvale,43.693781,-79.428191
62,York,"George Harvey Collegiate Institute,Runnymede C...","Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013


### 1.2 Create a map to show the neighbourhoods with schools

In [25]:
!pip install folium
!pip install geopy

import json
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim
import folium

from sklearn.cluster import KMeans

import requests

print('Libraries imported.')

Libraries imported.


In [26]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Amy")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [27]:
# create map of Toronto using latitude and longitude values

map = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    
map

### 1.3 Explore neibourhoods in Toronto and identify the ones with cafe being the most common venues

In [55]:
# The code was removed by Watson Studio for sharing.

In [56]:
# The code was removed by Watson Studio for sharing.

In [57]:
# The code was removed by Watson Studio for sharing.

Parkview Hill, Woodbine Gardens
Woodbine Heights
Thorncliffe Park
East Toronto, Broadview North (Old East York)
Leaside
Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West
The Kingsway, Montgomery Road, Old Mill North
South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
New Toronto, Mimico South, Humber Bay Shores
Alderwood, Long Branch
Islington Avenue, Humber Valley Village
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Northwest, West Humber - Clairville
Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East
Westmount
Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens
Glencairn
Humber Summit
Bedford Park, Lawrence Manor East
Don Mills
Willowdale, Willowdale West
Hillcrest Village
Fairview, Henry Farm,

In [58]:
V1=Venues.groupby('Neighbourhood').count()
V1

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Caledonia-Fairbanks,4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12


In [59]:
print('There are {} uniques categories.'.format(len(Venues['Venue Category'].unique())))

There are 142 uniques categories.


In [76]:
T_onehot = pd.get_dummies(Venues[['Venue Category']], prefix="", prefix_sep = "")
T_onehot['Neighbourhood'] = Venues['Neighbourhood']

fixed_columns = [T_onehot.columns[-1]] + list(T_onehot.columns[:-1])

T_onehot = T_onehot[fixed_columns]

T_onehot

,Neighbourhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,...,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,"Parkview Hill, Woodbine Gardens",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Parkview Hill, Woodbine Gardens",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Parkview Hill, Woodbine Gardens",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Parkview Hill, Woodbine Gardens",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Parkview Hill, Woodbine Gardens",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
454,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
455,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
456,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
# Delete the neighbourhoods without a gym or fitness centre
indexNames = T_onehot[(T_onehot['Gym'] == 0) & (T_onehot['Gym / Fitness Center'] == 0)].index
T_onehot.drop(indexNames,inplace=True)

In [81]:
for col in T_onehot:
    print(col)

Neighbourhood
Accessories Store
Airport
American Restaurant
Art Gallery
Arts & Crafts Store
Asian Restaurant
Athletics & Sports
Auto Garage
Bakery
Bank
Bar
Baseball Field
Beer Store
Bike Shop
Boutique
Breakfast Spot
Brewery
Bridal Shop
Bubble Tea Shop
Burger Joint
Burrito Place
Bus Line
Bus Station
Business Service
Butcher
Café
Caribbean Restaurant
Chinese Restaurant
Chocolate Shop
Clothing Store
Coffee Shop
College Stadium
Comfort Food Restaurant
Construction & Landscaping
Convenience Store
Curling Ice
Dance Studio
Deli / Bodega
Department Store
Dessert Shop
Dim Sum Restaurant
Diner
Discount Store
Dog Run
Drugstore
Electronics Store
Event Space
Fast Food Restaurant
Field
Fish & Chips Shop
Food & Drink Shop
Food Court
Food Truck
French Restaurant
Fried Chicken Joint
Frozen Yogurt Shop
Furniture / Home Store
Garden Center
Gas Station
Gastropub
General Entertainment
Gift Shop
Golf Course
Greek Restaurant
Grocery Store
Gym
Gym / Fitness Center
Hakka Restaurant
Hardware Store
Hobby Shop
Ho

In [82]:
df_g = T_onehot.groupby('Neighbourhood').mean().reset_index()
df_g.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,...,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Don Mills,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Downsview,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Mimico NW, The Queensway West, South of Bloor,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"New Toronto, Mimico South, Humber Bay Shores",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
df_g.shape

(8, 143)

In [85]:
num_top_venues = 10

for hood in df_g['Neighbourhood']:
    
    print("----"+hood+"----")
    temp = df_g[df_g['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood, Long Branch----
                       venue  freq
0                        Gym   1.0
1          Accessories Store   0.0
2               Noodle House   0.0
3         Mexican Restaurant   0.0
4  Middle Eastern Restaurant   0.0
5         Miscellaneous Shop   0.0
6          Mobile Phone Shop   0.0
7                      Motel   0.0
8              Movie Theater   0.0
9                       Park   0.0


----Don Mills----
                       venue  freq
0                        Gym   1.0
1          Accessories Store   0.0
2               Noodle House   0.0
3         Mexican Restaurant   0.0
4  Middle Eastern Restaurant   0.0
5         Miscellaneous Shop   0.0
6          Mobile Phone Shop   0.0
7                      Motel   0.0
8              Movie Theater   0.0
9                       Park   0.0


----Downsview----
                       venue  freq
0       Gym / Fitness Center   1.0
1          Accessories Store   0.0
2               Noodle House   0.0
3         Mexican R

In [86]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [87]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = df_g['Neighbourhood']

for ind in np.arange(df_g.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_g.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,"Alderwood, Long Branch",Gym,Yoga Studio,Drugstore
1,Don Mills,Gym,Yoga Studio,Drugstore
2,Downsview,Gym / Fitness Center,Yoga Studio,Drugstore
3,"Mimico NW, The Queensway West, South of Bloor,...",Gym,Yoga Studio,Drugstore
4,"New Toronto, Mimico South, Humber Bay Shores",Gym,Yoga Studio,Drugstore


### 1.4 Cluster neighbourhoods

In [88]:
kclusters = 5

df_g_clustering = df_g.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_g_clustering)

kmeans.labels_[0:10] 

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.


array([0, 0, 1, 0, 0, 1, 0, 2], dtype=int32)

In [89]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [90]:
df4 = df3.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

df4.head() 

,Borough,Name,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,East York,"East York Collegiate Institute,Leaside High Sc...","Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1.0,Gym / Fitness Center,Yoga Studio,Drugstore
1,East York,"East York Collegiate Institute,Leaside High Sc...",Woodbine Heights,43.695344,-79.318389,NaN,NaN,NaN,NaN
2,East York,"East York Collegiate Institute,Leaside High Sc...",Thorncliffe Park,43.705369,-79.349372,2.0,Gym / Fitness Center,Gym,Yoga Studio
3,East York,"East York Collegiate Institute,Leaside High Sc...","East Toronto, Broadview North (Old East York)",43.685347,-79.338106,NaN,NaN,NaN,NaN
4,East York,"East York Collegiate Institute,Leaside High Sc...",Leaside,43.709060,-79.363452,NaN,NaN,NaN,NaN


In [93]:
c1=df4.loc[df4['Cluster Labels']==0.0]
c1

,Borough,Name,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
5,Etobicoke,"Etobicoke Collegiate Institute,Kipling Collegi...","Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,0.0,Gym,Yoga Studio,Drugstore
9,Etobicoke,"Etobicoke Collegiate Institute,Kipling Collegi...","New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,0.0,Gym,Yoga Studio,Drugstore
10,Etobicoke,"Etobicoke Collegiate Institute,Kipling Collegi...","Alderwood, Long Branch",43.602414,-79.543484,0.0,Gym,Yoga Studio,Drugstore
20,North York,"A. Y. Jackson Secondary School,C. W. Jefferys ...",Don Mills,43.725900,-79.340923,0.0,Gym,Yoga Studio,Drugstore
37,North York,"A. Y. Jackson Secondary School,C. W. Jefferys ...",Don Mills,43.745906,-79.352188,0.0,Gym,Yoga Studio,Drugstore
48,Scarborough,"Agincourt Collegiate Institute,Albert Campbell...","Steeles West, L'Amoreaux West",43.799525,-79.318389,0.0,Gym,Yoga Studio,Drugstore


In [94]:
c2=df4.loc[df4['Cluster Labels']==1.0]
c2.head()

,Borough,Name,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,East York,"East York Collegiate Institute,Leaside High Sc...","Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1.0,Gym / Fitness Center,Yoga Studio,Drugstore
25,North York,"A. Y. Jackson Secondary School,C. W. Jefferys ...",Downsview,43.739015,-79.506944,1.0,Gym / Fitness Center,Yoga Studio,Drugstore
26,North York,"A. Y. Jackson Secondary School,C. W. Jefferys ...",Downsview,43.728496,-79.495697,1.0,Gym / Fitness Center,Yoga Studio,Drugstore
28,North York,"A. Y. Jackson Secondary School,C. W. Jefferys ...",Downsview,43.761631,-79.520999,1.0,Gym / Fitness Center,Yoga Studio,Drugstore
31,North York,"A. Y. Jackson Secondary School,C. W. Jefferys ...",Downsview,43.737473,-79.464763,1.0,Gym / Fitness Center,Yoga Studio,Drugstore


In [95]:
c3=df4.loc[df4['Cluster Labels']==2.0]
c3

,Borough,Name,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
2,East York,"East York Collegiate Institute,Leaside High Sc...",Thorncliffe Park,43.705369,-79.349372,2.0,Gym / Fitness Center,Gym,Yoga Studio


In [96]:
c4=df4.loc[df4['Cluster Labels']==3.0]
c4

,Borough,Name,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue


In [97]:
# We want to plot c2 because apparently people in neighbourhoods in c2 visits local resturants and cafe very offen
map_c2 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(c2['Latitude'], c2['Longitude'], c2['Borough'], c2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_c2)  
    
map_c2